# Ettoday 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例：

1. 取出今天所有的新聞
2. 取出現在時間兩小時內的新聞
3. 根據範例，取出三天前下午三點到五點的新聞

## 1. 取出今天所有的新聞

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup

browser = webdriver.Chrome(executable_path='chromedriver')

browser.get("https://www.ettoday.net/news/news-list.htm")

In [2]:
# 取得昨天時間
from datetime import datetime, timedelta
date_time = datetime.now() - timedelta(1)
date_time = date_time.strftime("%Y/%m/%d")
print("取得昨天時間：", date_time)

取得昨天時間： 2020/04/08


In [3]:
# 利用 selenium 持續執行下滑的動作，直到取得「昨天時間」的新聞，表示今天的已經取得完畢
import time
while True:
    browser.execute_script("window.scrollTo(0, 100000);")
    time.sleep(2)
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "html5lib")
    print(soup.find_all(class_="part_list_2")[-1].find_all('h3')[-1].find(class_="date").text)
    if date_time in soup.find_all(class_="part_list_2")[-1].find_all('h3')[-1].find(class_="date").text:
        print('==== STOP ====')
        break

2020/04/09 07:07
2020/04/09 06:27
2020/04/09 05:53
2020/04/09 00:45
2020/04/09 00:00
2020/04/08 23:22
==== STOP ====


In [4]:
# 取得資料
html_source = browser.page_source
soup = BeautifulSoup(html_source, "html5lib")

for d in soup.find(class_="part_list_2").find_all('h3'):
    if date_time in d.find(class_="date").text :
        break
    print(d.find(class_="date").text, d.find_all('a')[-1].text)

2020/04/09 10:26 汪喵狩獵誰強？一張圖「秒懂戰力差別」...致命一擊要靠牠
2020/04/09 10:24 雲端最前線／「學姊下凡來解惑」北市超前部署18招
2020/04/09 10:23 快訊／日本愛知縣明自行宣布「緊急狀態」！　籲755萬縣民避免外出
2020/04/09 10:23 口罩14天9片登場！台南清晨現人龍：同樣排隊卻有厚厚一包...更有安全感
2020/04/09 10:22 日常辦公暗藏感染風險...北京出招了：聚餐不要聊天！
2020/04/09 10:21 母親節回家去！高鐵「加開25班車」4/11開搶　5/8到5/11停售自由座
2020/04/09 10:21 譚德塞委屈扯台灣「對黑人種族歧視」！　阿滴173字罕見暴怒：到底在公三小
2020/04/09 10:21 口罩排隊人潮增4倍！藥師累癱大罵：超前部署，但裝備嚴重落後啊
2020/04/09 10:20 譚德塞3分鐘痛罵「台灣人身攻擊」　僑委會：僑界非常不以為然
2020/04/09 10:20 擔憂申請「藝文紓困」被退件！鄭麗君：盡可能從寬認定
2020/04/09 10:19 「有人說我是黑鬼」譚德塞狂罵台灣3分鐘　醫：一定要小心！別輕忽種族主義
2020/04/09 10:18 立委呼籲發租金補貼助勞工　勞長許銘春：目前並未考慮
2020/04/09 10:17 各國搶尋藥物救疫情　醫療保健類股中長線看好
2020/04/09 10:16 東貝董事長及發言人遭收押　預計召開臨時董事會因應
2020/04/09 10:11 零售業要倒光了！王浩宇的除臭襪「每月虧損幾十萬」　急喊紓困要快一點
2020/04/09 10:10 華為推出HUAWEI Card　功能致敬Apple Card
2020/04/09 10:10 CBA／林書豪歸隊合練高呼「我愛籃球」　期待復賽「珍惜任何小事」
2020/04/09 10:10 日職／球員工會號召對抗病毒　募款突破5000萬
2020/04/09 10:07 澳媒問「誰阻止發布全球公衛緊急事件」　WHO以隱私為由拒回應
2020/04/09 10:07 不分單雙號「一次領9片」首日！藥局狂湧3倍人潮　民眾一看驚呆
2020/04/09 10:06 國安基金績效下周一全都露？　財長：從不以賺錢為目的
2020/04/09 

## 2. 取出現在時間兩小時內的新聞

In [5]:
# 取得兩小時內的時間
from datetime import datetime, timedelta
two_hours_ago_time = datetime.now() - timedelta(hours=2)

# 取得資料（整理成好的格式）
news = []

for d in soup.find(class_="part_list_2").find_all('h3'):
    # 比兩小時前早的話就結束
    if datetime.strptime(d.find(class_="date").text, '%Y/%m/%d %H:%M') < two_hours_ago_time:
        break
    
    news.append({
        '時間': d.find(class_="date").text,
        '標題': d.find_all('a')[-1].text
    })

import pandas as pd
pd.DataFrame(news)

,時間,標題
0,2020/04/09 10:26,汪喵狩獵誰強？一張圖「秒懂戰力差別」...致命一擊要靠牠
1,2020/04/09 10:24,雲端最前線／「學姊下凡來解惑」北市超前部署18招
2,2020/04/09 10:23,快訊／日本愛知縣明自行宣布「緊急狀態」！ 籲755萬縣民避免外出
3,2020/04/09 10:23,口罩14天9片登場！台南清晨現人龍：同樣排隊卻有厚厚一包...更有安全感
4,2020/04/09 10:22,日常辦公暗藏感染風險...北京出招了：聚餐不要聊天！
5,2020/04/09 10:21,母親節回家去！高鐵「加開25班車」4/11開搶 5/8到5/11停售自由座
6,2020/04/09 10:21,譚德塞委屈扯台灣「對黑人種族歧視」！ 阿滴173字罕見暴怒：到底在公三小
7,2020/04/09 10:21,口罩排隊人潮增4倍！藥師累癱大罵：超前部署，但裝備嚴重落後啊
8,2020/04/09 10:20,譚德塞3分鐘痛罵「台灣人身攻擊」 僑委會：僑界非常不以為然
9,2020/04/09 10:20,擔憂申請「藝文紓困」被退件！鄭麗君：盡可能從寬認定


## 3. 根據範例，取出三天前下午三點到五點的新聞

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup

browser = webdriver.Chrome(executable_path='chromedriver')

browser.get("https://www.ettoday.net/news/news-list.htm")

In [2]:
# 取得三天前下午三點到五點的時間
from datetime import datetime, timedelta

date_time = datetime.now() - timedelta(3)
print("取得昨天時間：", date_time)

date_time_month = str(int(date_time.strftime("%m")))
date_time_day = str(int(date_time.strftime("%d")))
print("取得昨天時間（月）：", date_time_month)
print("取得昨天時間（日）：", date_time_day)


three_days_15_clock = datetime.strptime(date_time.strftime("%Y/%m/%d") + ' 15', '%Y/%m/%d %H')
three_days_17_clock = datetime.strptime(date_time.strftime("%Y/%m/%d") + ' 17', '%Y/%m/%d %H')
print("取得昨天下午三點：", three_days_15_clock)
print("取得昨天下午五點：", three_days_17_clock)

取得昨天時間： 2020-04-06 10:31:31.450468
取得昨天時間（月）： 4
取得昨天時間（日）： 6
取得昨天下午三點： 2020-04-06 15:00:00
取得昨天下午五點： 2020-04-06 17:00:00


In [3]:
# 選擇且點選日期
from selenium.webdriver.support.ui import Select

selM = Select(browser.find_element_by_id("selM"))
selM.select_by_value(date_time_month)
selD = Select(browser.find_element_by_id("selD"))
selD.select_by_value(date_time_day)
browser.find_element_by_id('button').click()

In [5]:
# 利用 selenium 持續執行下滑的動作，直到取得要求時間內的新聞，表示取得完畢
import time
while True:
    browser.execute_script("window.scrollTo(0, 100000);")
    time.sleep(1)
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "html5lib")
    new_date = soup.find_all(class_="part_list_2")[-1].find_all('h3')[-1].find(class_="date").text
    print(new_date)
    if datetime.strptime(new_date, '%Y/%m/%d %H:%M') < three_days_15_clock:
        print('==== STOP ====')
        break

2020/04/06 18:41
2020/04/06 18:41
2020/04/06 18:41
2020/04/06 18:41
2020/04/06 18:41
2020/04/06 18:41
2020/04/06 18:25
2020/04/06 18:11
2020/04/06 18:00
2020/04/06 17:51
2020/04/06 17:42
2020/04/06 17:32
2020/04/06 17:25
2020/04/06 17:15
2020/04/06 17:07
2020/04/06 16:58
2020/04/06 16:51
2020/04/06 16:42
2020/04/06 16:35
2020/04/06 16:27
2020/04/06 16:18
2020/04/06 16:11
2020/04/06 15:58
2020/04/06 15:49
2020/04/06 15:32
2020/04/06 15:23
2020/04/06 15:15
2020/04/06 15:07
2020/04/06 15:00
2020/04/06 14:50
==== STOP ====


In [6]:
# 取得資料（整理成好的格式）
news = []

for d in soup.find(class_="part_list_2").find_all('h3'):
    # 比兩小時前早的話就結束
    if datetime.strptime(d.find(class_="date").text, '%Y/%m/%d %H:%M') < three_days_15_clock:
        print('==== STOP ====')
        break
    
    if datetime.strptime(d.find(class_="date").text, '%Y/%m/%d %H:%M') < three_days_17_clock:
        news.append({
            '時間': d.find(class_="date").text,
            '標題': d.find_all('a')[-1].text
        })

import pandas as pd
pd.DataFrame(news)

==== STOP ====


,時間,標題
0,2020/04/06 16:59,蔡英文「視訊接見」美國密涅瓦大學生 狂讚國家隊、捐口罩與國際共度難關
1,2020/04/06 16:59,佛心建商發2萬防疫金給員工 透露「2個月後再研擬是否加發」
2,2020/04/06 16:58,酷碰券衝內需 沈榮津：可望帶動十倍消費、上看千億元商機
3,2020/04/06 16:58,新冠肺炎新症狀！ 紐約返回女唯一病徵「大便次數明顯增加」確診
4,2020/04/06 16:58,深夜遇警盤查...她夾緊緊急喊：我沒前科！一個深蹲菊花掉出「4包毒」
5,2020/04/06 16:57,S10、Note 10軟體更新推播 「一鍵拍錄」正式下放！
6,2020/04/06 16:56,中職／富邦開幕戰洋投擇一先發 包林傑要等到5月中
7,2020/04/06 16:56,羅友志／誰在權鬥？警政、觀光人事鬧雙重標準！
8,2020/04/06 16:55,最毒班機全機人數曝光！3/30紐約飛台北機上9人確診全是乘客
9,2020/04/06 16:54,求子多年才懷孕！她「婉拒飯局」在家防疫…遭閨蜜怒罵：妳懷龍種喔
